In [75]:
import json
class data_generator:
    def __init__(self, data, batch_size=1):
        self.data = data
        self.batch_size = batch_size
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1

    def __len__(self):
        return self.steps

    def __iter__(self):
        while True:
            idxs = list(range(len(self.data)))
            np.random.shuffle(idxs)
            T, S1, S2, K1, K2, O1, O2, = [], [], [], [], [], [], []
            for i in idxs:
                d = self.data[i]
                text = d['text']
                items = {}
                print(text)
                print(i)
                for sp in d['spo_list']:
                    subjectid = text.find(sp[0])
                    objectid = text.find(sp[2])
                    if subjectid != -1 and objectid != -1:
                        key = (subjectid, subjectid + len(sp[0]))
                        # 这里相同subject做key时只会取第一个spo
                        if key not in items:
                            items[key] = []
                        items[key].append((objectid,
                                           objectid + len(sp[2]),
                                           predicate2id[sp[1]]))
                print(items)
                if items:
                    T.append([char2id.get(c, 1) for c in text])  # 1是unk，0是padding
                    s1, s2 = [0] * len(text), [0] * len(text)
                    for j in items:
                        s1[j[0]] = 1
                        s2[j[1] - 1] = 1
                    k1, k2 = choice(list(items.keys()))
                    o1, o2 = [0] * len(text), [0] * len(text)  # 0是unk类（共49+1个类）
                    for j in items[(k1, k2)]:
                        o1[j[0]] = j[2]
                        o2[j[1] - 1] = j[2]
                    S1.append(s1)
                    S2.append(s2)
                    K1.append([k1])
                    K2.append([k2 - 1])
                    O1.append(o1)
                    O2.append(o2)
                    if len(T) == self.batch_size or i == idxs[-1]:
                        T = np.array(seq_padding(T))
                        S1 = np.array(seq_padding(S1))
                        S2 = np.array(seq_padding(S2))
                        O1 = np.array(seq_padding(O1))
                        O2 = np.array(seq_padding(O2))
                        K1, K2 = np.array(K1), np.array(K2)
                        yield [T, S1, S2, K1, K2, O1, O2], None
                        T, S1, S2, K1, K2, O1, O2, = [], [], [], [], [], [], []

In [39]:
train_data = json.load(open('../data_processed/train_data_me.json'))

In [76]:
ds = data_generator(train_data)

In [77]:
g = ds.__iter__()

In [15]:
import numpy as np

In [79]:
train_data[164629]

{'text': '《再见.天堂》由新生代音乐人夏炎独自诠释自己的创作，运用木吉他跟手铃的搭配，平铺直述的用最单纯的声音唱出自己的内心世界',
 'spo_list': [['再见.天堂', '作曲', '夏炎']]}

In [78]:
a = next(g)

《再见.天堂》由新生代音乐人夏炎独自诠释自己的创作，运用木吉他跟手铃的搭配，平铺直述的用最单纯的声音唱出自己的内心世界
164629
{(1, 6): [(14, 16, 47)]}


In [84]:
a[0][4][0]

array([5])

In [20]:
dev_data = json.load(open('../data_processed/dev_data_me.json'))
id2predicate, predicate2id = json.load(open('../data_processed/all_50_schemas_me.json'))
id2predicate = {int(i): j for i, j in id2predicate.items()}
id2char, char2id = json.load(open('../data_processed/all_chars_me.json'))

In [27]:
from random import choice

In [32]:
def seq_padding(X):
    L = [len(x) for x in X]
    ML = max(L)
    return [x + [0] * (ML - len(x)) for x in X]